## Extreme Weather Events

This notebook trains a RNN on HURdat Extreme Weather Events.

We first start by preprocessing the data

In [15]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import pandas as pd
import math
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split

In [16]:
df = pd.read_csv('data/HURdat_ExtremeWeatherEvents.csv',index_col=0)

In [17]:
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found.')

No GPU found.


In [18]:
print(df.shape)
print(df.columns)

(8793, 21)
Index(['ID', 'Name', 'Status', 'Latitude', 'Longitude', 'Maximum.Wind',
       'date_time', 'diff', 'rapid_int', 'i', 'n', 'persistence', 'product',
       'Initial.Max', 'speed', 'speed_z', 'speed_m', 'Jday', 'Maximum.Wind_p',
       'Latitude_p', 'Longitude_p'],
      dtype='object')


In [19]:
event_ids = df['ID'].unique().tolist()
num_events = len(event_ids)

In [20]:
sequence_length = 4
prediction_window = 4
batch_size = 10
# sequence_size : 4 to cover a whole day
# prediction_window : how many time steps in the future we predict for.

In [21]:
columns_to_drop = ['Name','Status','date_time','i','n','Jday']
columns_to_scale = ['Latitude','Longitude','Maximum.Wind','diff',
           'persistence','product','Initial.Max','speed','speed_z',
           'speed_m','Maximum.Wind_p','Latitude_p', 'Longitude_p']
df = df.drop(columns_to_drop,axis=1)
df.replace({np.nan: 0}, inplace=True)

In [22]:
test = df[df['ID']=='AL011980']

In [23]:
test.iloc[8]

ID                 AL011980
Latitude               29.6
Longitude             -85.8
Maximum.Wind             25
diff                      5
rapid_int                 0
persistence               0
product                   0
Initial.Max              25
speed               13.7483
speed_z           0.0345869
speed_m             9.26624
Maximum.Wind_p           20
Latitude_p             29.5
Longitude_p           -84.7
Name: 11, dtype: object

In [24]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_scaled = df.copy(deep=True)
df_scaled[columns_to_scale] = scaler.fit_transform(df_scaled[columns_to_scale])

In [25]:
test = df_scaled[df_scaled['ID']=='AL011980']

## Batches and data loaders

In [26]:
def get_batches(df_data):
    x_batch, y_batch = [],[]
    for event_id in event_ids:
        event = df_data[df_data['ID']==event_id]
        for i in range(1,event.shape[0]-sequence_length-prediction_window):
            x = np.array(event.iloc[i:i+sequence_length].drop('ID',axis=1),dtype=np.double)
            y = np.array(event.iloc[i+sequence_length+prediction_window-1]['rapid_int'],dtype=np.double)
            x_batch.append(x)
            y_batch.append(y)
    return np.array(x_batch,dtype=np.double), np.array(y_batch,dtype=np.long)

In [27]:
def get_dataloader(df_data, batch_size):
    x,y = get_batches(df_data)
    data = TensorDataset(torch.from_numpy(x), torch.from_numpy(y))
    data_loader = DataLoader(data, shuffle=True, batch_size=batch_size)
    return data_loader

In [28]:
test_loader = get_dataloader(df_scaled,5)

data_iter = iter(test_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([5, 4, 14])
tensor([[[-0.4008,  0.5283, -0.8139, -0.6147,  0.0000, -0.7340, -0.5517,
           0.2061, -0.2609, -0.0724,  0.2234, -0.6803, -0.1992,  0.3502],
         [-0.4008,  0.4777, -0.8139, -0.6147,  0.0000, -0.7340, -0.5517,
           0.2061, -0.2876, -0.0838,  0.2234, -0.6803, -0.2100,  0.3116],
         [-0.4130,  0.4322, -0.8139, -0.6147,  0.0000, -0.7340, -0.5517,
           0.2061, -0.3056, -0.0838,  0.2234, -0.6803, -0.2100,  0.2688],
         [-0.4252,  0.3766, -0.8139, -0.6147,  0.0000, -0.7340, -0.5517,
           0.2061, -0.3055, -0.0952,  0.2234, -0.6803, -0.2209,  0.2302]],

        [[-1.1935,  0.9174, -0.8139, -0.1951,  0.0000,  0.0315, -0.3036,
          -0.7668, -0.2329, -1.1177,  0.2234, -0.6803, -0.9806,  0.6887],
         [-1.1081,  0.8719, -0.8139, -0.1951,  0.0000,  0.0315, -0.3036,
          -0.7668, -0.2378, -1.0498,  0.2234, -0.6803, -0.9155,  0.6415],
         [-1.0349,  0.8366, -0.8139, -0.1951,  0.0000, -0.7340, -0.5517,
          -0.7668, -

In [29]:
for i in range(1,test.shape[0]-sequence_length):
    print('DataFrame slice : {} , {} \t Predictor index : {}'.format(i,i+sequence_length-1,i+sequence_length+prediction_window-1))

DataFrame slice : 1 , 4 	 Predictor index : 8
DataFrame slice : 2 , 5 	 Predictor index : 9
DataFrame slice : 3 , 6 	 Predictor index : 10
DataFrame slice : 4 , 7 	 Predictor index : 11
DataFrame slice : 5 , 8 	 Predictor index : 12
DataFrame slice : 6 , 9 	 Predictor index : 13
DataFrame slice : 7 , 10 	 Predictor index : 14
DataFrame slice : 8 , 11 	 Predictor index : 15
DataFrame slice : 9 , 12 	 Predictor index : 16


### Train test split

In [30]:
test_ratio = 0.2

event_ids = df['ID'].unique().tolist()
training_ids, test_ids = train_test_split(event_ids, test_size=test_ratio, random_state=42)

#should split training into training and validaton.

df_train = df_scaled[df_scaled['ID'].isin(training_ids)]
df_test = df_scaled[df_scaled['ID'].isin(test_ids)]

print('Training dataframe : {}'.format(df_train.shape))
print('Testing dataframe : {}'.format(df_test.shape))


Training dataframe : (7179, 15)
Testing dataframe : (1614, 15)


## Model

Next we define the LSTM Model

In [31]:
class WeatherRNN(nn.Module):
    
    def __init__(self, n_hidden=256, n_layers=2, drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        self.lstm = nn.LSTM(df_scaled.shape[1]-1, n_hidden, n_layers, dropout=drop_prob, batch_first=True)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(n_hidden, 2)
      
    def forward(self, x, hidden):
        batch_size = x.size(0)
        lstm_out, hidden = self.lstm(x, hidden)
        lstm_out = self.dropout(lstm_out)
        lstm_out = lstm_out.contiguous().view(-1, self.n_hidden)
        output = self.fc(lstm_out)
        output = output.view(batch_size, -1, 2)
        out = output[:, -1]
        return out, hidden
       
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        if (train_on_gpu):
            hidden = (weight.new_zeros((self.n_layers, batch_size, self.n_hidden),dtype=torch.float64).cuda(),
                      weight.new_zeros((self.n_layers, batch_size, self.n_hidden),dtype=torch.float64).cuda())
        else:
            hidden = (weight.new_zeros((self.n_layers, batch_size, self.n_hidden),dtype=torch.float64),
                      weight.new_zeros((self.n_layers, batch_size, self.n_hidden),dtype=torch.float64))
        return hidden

In [32]:
model = WeatherRNN()
model.double()
print(model)

WeatherRNN(
  (lstm): LSTM(14, 256, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=256, out_features=2, bias=True)
)


## Training

Next we define the training loop.

In [33]:
def train(model, epochs, batch_size, lr):
    model.train()
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()    
    
    if(train_on_gpu):
        model.cuda()
    
    for e in range(epochs):
        h = model.init_hidden(batch_size)
        losses = []
        for inputs, targets in train_loader:
            if inputs.size(0) != batch_size:
                break
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()
            h = tuple([each.data for each in h])
            model.zero_grad()
            output, h = model(inputs, h)
            loss = criterion(output, targets)
            losses.append(loss.item())
            loss.backward()
            optimizer.step()
        print("Epoch: \t {} of {} ...".format(e+1, epochs))
        print("Loss: \t {:.4f}".format(np.mean(losses)))
        print("*****************")
            #if counter % print_every == 0:
            #    calculate_loss(model, validation_data, batch_size, seq_length)
            #    model.train()
            


In [34]:
epochs = 50
batch_size = 10
lr = 0.001

In [35]:
train_loader = get_dataloader(df_train, batch_size)
test_loader = get_dataloader(df_test, batch_size)

In [36]:
train(model, epochs=epochs, batch_size=batch_size, lr=lr)

Epoch: 	 1 of 100 ...
Loss: 	 0.3048
*****************
Epoch: 	 2 of 100 ...
Loss: 	 0.2750
*****************
Epoch: 	 3 of 100 ...
Loss: 	 0.2724
*****************
Epoch: 	 4 of 100 ...
Loss: 	 0.2720
*****************
Epoch: 	 5 of 100 ...
Loss: 	 0.2692
*****************
Epoch: 	 6 of 100 ...
Loss: 	 0.2628
*****************
Epoch: 	 7 of 100 ...
Loss: 	 0.2623
*****************
Epoch: 	 8 of 100 ...
Loss: 	 0.2624
*****************
Epoch: 	 9 of 100 ...
Loss: 	 0.2569
*****************
Epoch: 	 10 of 100 ...
Loss: 	 0.2558
*****************
Epoch: 	 11 of 100 ...
Loss: 	 0.2567
*****************
Epoch: 	 12 of 100 ...
Loss: 	 0.2518
*****************
Epoch: 	 13 of 100 ...
Loss: 	 0.2527
*****************
Epoch: 	 14 of 100 ...
Loss: 	 0.2534
*****************
Epoch: 	 15 of 100 ...
Loss: 	 0.2495
*****************
Epoch: 	 16 of 100 ...
Loss: 	 0.2461
*****************
Epoch: 	 17 of 100 ...
Loss: 	 0.2433
*****************
Epoch: 	 18 of 100 ...
Loss: 	 0.2464
*****************
E

## Test the model

In [37]:
test_losses = []
num_correct = 0

h = model.init_hidden(batch_size)
criterion = nn.CrossEntropyLoss()
    
model.eval()

nb_classes = 2
confusion_matrix = torch.zeros(nb_classes, nb_classes)

for inputs, labels in test_loader:

    if inputs.size(0) != batch_size:
        break
    h = tuple([each.data for each in h])
    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    output, h = model(inputs, h)
    test_loss = criterion(output, labels)
    
    softmax = nn.Softmax(dim=1)
    output = softmax(output)
    _, pred = torch.max(output, 1)  
    
    test_losses.append(test_loss.item())
    correct_pred_tensor = pred.eq(labels.data.view_as(pred))

    for t, p in zip(labels.view(-1), pred.view(-1)):
        confusion_matrix[t.long(), p.long()] += 1
    
    correct = np.squeeze(correct_pred_tensor.numpy()) if not train_on_gpu \
            else np.squeeze(correct_pred_tensor.cpu().numpy())
    num_correct += np.sum(correct)

print("Test loss: {:.3f}".format(np.mean(test_losses)))
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))



Test loss: 1.019
Test accuracy: 0.853


In [38]:

print("Test loss: {:.3f}".format(np.mean(test_losses)))

# accuracy over all test data
test_acc = num_correct/len(test_loader.dataset)
print("Test accuracy: {:.3f}".format(test_acc))

print('\nConfusion Matrix: {}\n'.format(confusion_matrix))
percision = confusion_matrix[1][1] / sum(confusion_matrix[1])
print('Percision: {}'.format(percision))
recall = confusion_matrix[1][1] / (confusion_matrix[1][1]+confusion_matrix[0][0])
print('Recall: {}'.format(recall))
f1 = 2*percision*recall/percision+recall
print('f1 score: {}'.format(f1))

Test loss: 1.019
Test accuracy: 0.853

Confusion Matrix: tensor([[629.,  42.],
        [ 61.,   8.]])

Percision: 0.11594203114509583
Recall: 0.012558870017528534
f1 score: 0.0376766100525856


In [39]:
confusion_matrix[1]

tensor([61.,  8.])

# Predict location

In [40]:
df = pd.read_csv('data/HURdat_ExtremeWeatherEvents.csv',index_col=0)

In [41]:
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found.')

No GPU found.


In [42]:
print(df.shape)
print(df.columns)

(8793, 21)
Index(['ID', 'Name', 'Status', 'Latitude', 'Longitude', 'Maximum.Wind',
       'date_time', 'diff', 'rapid_int', 'i', 'n', 'persistence', 'product',
       'Initial.Max', 'speed', 'speed_z', 'speed_m', 'Jday', 'Maximum.Wind_p',
       'Latitude_p', 'Longitude_p'],
      dtype='object')


In [43]:
event_ids = df['ID'].unique().tolist()
num_events = len(event_ids)

In [44]:
sequence_length = 4
prediction_window = 4
batch_size = 10
# sequence_size : 4 to cover a whole day
# prediction_window : how many time steps in the future we predict for.

In [45]:
columns_to_drop = ['Name','Status','date_time','i','n','Jday']
columns_to_scale = ['rapid_int','Maximum.Wind','diff',
           'persistence','product','Initial.Max','speed','speed_z',
           'speed_m','Maximum.Wind_p','Latitude_p', 'Longitude_p']
df = df.drop(columns_to_drop,axis=1)
df.replace({np.nan: 0}, inplace=True)

In [46]:
test = df[df['ID']=='AL011980']

In [47]:
test.iloc[8]

ID                 AL011980
Latitude               29.6
Longitude             -85.8
Maximum.Wind             25
diff                      5
rapid_int                 0
persistence               0
product                   0
Initial.Max              25
speed               13.7483
speed_z           0.0345869
speed_m             9.26624
Maximum.Wind_p           20
Latitude_p             29.5
Longitude_p           -84.7
Name: 11, dtype: object

In [48]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_scaled = df.copy(deep=True)
df_scaled[columns_to_scale] = scaler.fit_transform(df_scaled[columns_to_scale])

In [56]:
def get_location_batches(df_data):
    x_batch, y_batch = [],[]
    for event_id in event_ids:
        event = df_data[df_data['ID']==event_id]
        for i in range(1,event.shape[0]-sequence_length-prediction_window):
            x = np.array(event.iloc[i:i+sequence_length].drop('ID',axis=1),dtype=np.double)
            y = np.array(event.iloc[i+sequence_length+prediction_window-1][['Latitude','Longitude']],dtype=np.double)
            x_batch.append(x)
            y_batch.append(y)
    return np.array(x_batch,dtype=np.double), np.array(y_batch,dtype=np.double)

In [57]:
def get_location_dataloader(df_data, batch_size):
    x,y = get_location_batches(df_data)
    data = TensorDataset(torch.from_numpy(x), torch.from_numpy(y))
    data_loader = DataLoader(data, shuffle=True, batch_size=batch_size)
    return data_loader

In [58]:
test_loader = get_location_dataloader(df_scaled,5)

data_iter = iter(test_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([5, 4, 14])
tensor([[[ 1.8900e+01, -6.6900e+01, -2.3831e-01, -1.9507e-01, -2.5630e-01,
           3.1463e-02, -1.3820e-01, -1.1820e-01, -1.4992e-01, -5.5251e-01,
           2.2341e-01,  5.5450e-02, -4.3795e-01, -2.6681e-01],
         [ 2.2700e+01, -7.3700e+01, -4.6446e-02,  2.2453e-01, -2.5630e-01,
           3.1463e-02, -1.3820e-01, -1.1820e-01, -1.7354e-01, -4.9714e-02,
           2.2341e-01, -3.1245e-01,  2.8711e-02, -5.8387e-01],
         [ 2.3700e+01, -7.4500e+01,  3.3728e-01,  1.4833e+00, -2.5630e-01,
           1.5625e+00,  6.8888e-01, -1.1820e-01, -2.5449e-01,  6.1599e-02,
           2.2341e-01,  5.5450e-02,  9.3827e-02, -6.2244e-01],
         [ 2.4800e+01, -7.4400e+01,  7.2100e-01,  2.3225e+00,  3.9017e+00,
           2.3280e+00,  1.4333e+00,  2.0611e-01, -2.6258e-01,  1.4812e-01,
           2.2341e-01,  4.2335e-01,  2.0235e-01, -6.5671e-01]],

        [[ 2.7900e+01, -7.4100e+01,  7.2100e-01, -1.9507e-01, -2.5630e-01,
          -7.3403e-01, -5.5174e-01,  1.1790e+00,

In [59]:
test_ratio = 0.2

event_ids = df['ID'].unique().tolist()
training_ids, test_ids = train_test_split(event_ids, test_size=test_ratio, random_state=42)

#should split training into training and validaton.

df_train = df_scaled[df_scaled['ID'].isin(training_ids)]
df_test = df_scaled[df_scaled['ID'].isin(test_ids)]

print('Training dataframe : {}'.format(df_train.shape))
print('Testing dataframe : {}'.format(df_test.shape))


Training dataframe : (7179, 15)
Testing dataframe : (1614, 15)


In [60]:
epochs = 50
batch_size = 10
lr = 0.001

In [61]:
train_loader = get_dataloader(df_train, batch_size)
test_loader = get_dataloader(df_test, batch_size)

In [62]:
def train(model, epochs, batch_size, lr):
    model.train()
    
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()    
    
    if(train_on_gpu):
        model.cuda()
    
    for e in range(epochs):
        h = model.init_hidden(batch_size)
        losses = []
        for inputs, targets in train_loader:
            if inputs.size(0) != batch_size:
                break
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()
            h = tuple([each.data for each in h])
            model.zero_grad()
            output, h = model(inputs, h)
            loss = criterion(output, targets)
            losses.append(loss.item())
            loss.backward()
            optimizer.step()
        print("Epoch: \t {} of {} ...".format(e+1, epochs))
        print("Loss: \t {:.4f}".format(np.mean(losses)))
        print("*****************")
            #if counter % print_every == 0:
            #    calculate_loss(model, validation_data, batch_size, seq_length)
            #    model.train()
            


In [63]:
train(model, epochs=epochs, batch_size=batch_size, lr=lr)

Epoch: 	 1 of 50 ...
Loss: 	 664.6868
*****************
Epoch: 	 2 of 50 ...
Loss: 	 121.6948
*****************
Epoch: 	 3 of 50 ...
Loss: 	 70.2506
*****************
Epoch: 	 4 of 50 ...
Loss: 	 54.4332
*****************
Epoch: 	 5 of 50 ...
Loss: 	 48.1410
*****************
Epoch: 	 6 of 50 ...
Loss: 	 43.5762
*****************
Epoch: 	 7 of 50 ...
Loss: 	 42.5632
*****************
Epoch: 	 8 of 50 ...
Loss: 	 41.5208
*****************
Epoch: 	 9 of 50 ...
Loss: 	 40.0970
*****************
Epoch: 	 10 of 50 ...
Loss: 	 41.1047
*****************
Epoch: 	 11 of 50 ...
Loss: 	 40.1529
*****************
Epoch: 	 12 of 50 ...
Loss: 	 39.1158
*****************
Epoch: 	 13 of 50 ...
Loss: 	 38.8028
*****************
Epoch: 	 14 of 50 ...
Loss: 	 38.5854
*****************
Epoch: 	 15 of 50 ...
Loss: 	 38.3142
*****************
Epoch: 	 16 of 50 ...
Loss: 	 38.0691
*****************
Epoch: 	 17 of 50 ...
Loss: 	 35.8702
*****************
Epoch: 	 18 of 50 ...
Loss: 	 37.4851
*****************

In [64]:
test_losses = []
num_correct = 0

h = model.init_hidden(batch_size)
criterion = nn.MSELoss()
    
model.eval()

for inputs, labels in test_loader:

    if inputs.size(0) != batch_size:
        break
    h = tuple([each.data for each in h])
    if(train_on_gpu):
        inputs, labels = inputs.cuda(), labels.cuda()
    output, h = model(inputs, h)
    test_loss = criterion(output, labels)    
    test_losses.append(test_loss.item())

print("Test accuracy: {:.3f}".format(test_acc))


Test accuracy: 0.853
